In [1]:
import pickle
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os
path = '/nfs/team283/aa16/KR_NAS/'
os.chdir(path)
sys.path.append(path)
from IPython.display import Image
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True' + ',dnn.enabled=False'
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

Construct AnnData object:

In [2]:
counts = pd.read_table('data/CRcells_14pcw/ns_tic1219_twobatch/az1219b2_TargetCountMatrix.txt')
genes = counts['TargetName']
counts = counts.drop('TargetName',1)
counts = counts.rename(index=genes)
counts = counts.astype('int')
metadata = pd.read_csv('data/CRcells_14pcw/NS_DSP human foetal brain Cajal-Retzius NanoString profiling 2 annotations.csv')
metadata = metadata.rename(index=metadata['Sanger Sample ID'])
print('Metadata, but no counts, number:', len([c for c in metadata.index if c not in np.array(counts.columns)]))
print('Metadata, but no counts, names:',[c for c in metadata.index if c not in np.array(counts.columns)])
print('Metadata and counts, number:', len([c for c in metadata.index if c in np.array(counts.columns)]))
print('Counts, but no metadata, number:', len([c for c in np.array(counts.columns) if c not in metadata.index]))
metadata = metadata.loc[~pd.isna(metadata.index),:]
metadata = metadata.reindex(np.array(counts.columns))
properties = pd.read_table('data/CRcells_14pcw/ns_tic1219_twobatch/az1219b2_SegmentProperties.txt')
properties = properties.rename(index=properties['Sample_ID'])
properties = properties.reindex(np.array(metadata.index))
properties = properties.drop(labels='Sample_ID', axis = 1)
probe_counts = pd.read_table('data/CRcells_14pcw/ns_tic1219_twobatch/az1219b2_BioProbeCountMatrix.txt')
genes = probe_counts['TargetName']
probe_counts = probe_counts.drop(['TargetName', 'ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol',
                                 'Accessions', 'GenomeBuild', 'GenomicPosition', 'GlobalOutlier', 'GlobalOutlierReason',
                                 'OutlierFrequency', 'OutlierAOIs', 'Gene'],1)
probe_counts = probe_counts.rename(index=genes)
probe_counts = probe_counts.astype('int')
negprobe_counts = probe_counts.loc[probe_counts.index == 'NegProbe-WTX', ]
negprobe_counts.index = negprobe_counts.index + '-' + [str(x) for x in range(len(negprobe_counts.index))]
adataB = sc.AnnData(X=np.array(counts).T, obs=pd.concat((metadata, properties), axis = 1), var=np.array(counts.index))
adataB.var_names = counts.index
adataB.obsm['negProbes'] = negprobe_counts.T
adataB.obs['GeoLOD2.0_01'] = adataB.obs['NegGeoMean_01'] * adataB.obs['NegGeoSD_01']**2
adataB.obs['DetectedGenes2.0'] = np.sum(np.array([adataB.X[i,:] >= adataB.obs['GeoLOD2.0_01'].iloc[i] for i in range(len(adataB.X[:,1]))]), axis = 1)
adataB.var['SYMBOL'] = adataB.var[0]
adataB.var = adataB.var.drop(0, axis = 1)
adataB.var.index = [str(x) for x in range(len(adataB.var.index))]
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 0] = 5
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 1] = 5
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 2] = 5
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 3] = 5
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 4] = 5

Metadata, but no counts, number: 42
Metadata, but no counts, names: ['HCA_F_BRA11360456', 'HCA_F_BRA11360459', 'HCA_F_BRA11360462', 'HCA_F_BRA11360465', 'HCA_F_BRA11360468', 'HCA_F_BRA11360471', 'HCA_F_BRA11360474', 'HCA_F_BRA11360477', 'HCA_F_BRA11360480', 'HCA_F_BRA11360483', 'HCA_F_BRA11360486', 'HCA_F_BRA11360489', 'HCA_F_BRA11360492', 'HCA_F_BRA11360495', 'HCA_F_BRA11360498', 'HCA_F_BRA11360501', 'HCA_F_BRA11360504', 'HCA_F_BRA11360507', 'HCA_F_BRA11360510', 'HCA_F_BRA11360513', 'HCA_F_BRA11360516', 'HCA_F_BRA11360519', 'HCA_F_BRA11360522', 'HCA_F_BRA11360525', 'HCA_F_BRA11360528', 'HCA_F_BRA11360531', 'HCA_F_BRA11360534', 'HCA_F_BRA11360537', 'HCA_F_BRA11360540', 'HCA_F_BRA11360543', 'HCA_F_BRA11360546', 'HCA_F_BRA11360549', 'HCA_F_BRA11360552', 'HCA_F_BRA11360555', 'HCA_F_BRA11360558', 'HCA_F_BRA11360561', 'HCA_F_BRA11360564', 'HCA_F_BRA11360567', 'HCA_F_BRA11360570', 'HCA_F_BRA11360573', 'PhiX', 'No match']
Metadata and counts, number: 81
Counts, but no metadata, number: 0


/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [3]:
adataB

AnnData object with n_obs × n_vars = 81 × 18677
    obs: 'Sample_ID', 'Manifest ID', 'slide name', 'scan name', 'panel', 'roi', 'segment', 'aoi', 'area', 'nuclei', 'Type', 'Species', 'Age', 'Background', 'Tissue', 'Slide code', 'Sample slide ID', 'TR', 'Cy5', 'Cy3', 'Pool', 'Collection plate', 'Well', 'PCR plate', 'PCR well', 'Primer plate', 'Primer well', 'I7_Index_ID', 'index', 'I5_Index_ID', 'index2', 'Notes', 'Sanger Sample ID', 'Tag', 'Reads, count', 'Reads, %', 'expected_neg', 'ns_sample_ID', 'RawReads', 'TrimmedReads', 'StitchedReads', 'AlignedReads', 'DeduplicatedReads', 'SequencingSaturation', 'UID', 'NegGeoMean_01', 'NegGeoSD_01', 'GeoLOQ2.5_01', 'NormFactorNeg_01', 'NormFactorQ3', 'NormFactorHK', 'GeoLOD2.0_01', 'DetectedGenes2.0'
    var: 'SYMBOL'
    obsm: 'negProbes'

In [4]:
counts = pd.read_table('data/ns_tic1106_outputs/Run2_TargetCountMatrix.txt')
genes = counts['TargetName']
counts = counts.drop('TargetName',1)
counts = counts.rename(index=genes)
counts = counts.astype('int')
metadata = pd.read_csv('data/NS_DSP CR annotations.csv')
metadata = metadata.rename(index=metadata['Sample_ID'])
print(len([c for c in metadata.index if c not in np.array(counts.columns)]))
print([c for c in metadata.index if c not in np.array(counts.columns)])
print(len([c for c in metadata.index if c in np.array(counts.columns)]))
print(len([c for c in np.array(counts.columns) if c not in metadata.index]))
metadata = metadata.loc[~pd.isna(metadata.index),:]
metadata = metadata.reindex(np.array(counts.columns))
properties = pd.read_table('data/ns_tic1106_outputs/Run2_SegmentProperties.txt')
properties = properties.rename(index=properties['Sample_ID'])
properties = properties.reindex(np.array(metadata.index))
properties = properties.drop(labels='Sample_ID', axis = 1)
probe_counts = pd.read_table('data/ns_tic1106_outputs/Run2_BioProbeCountMatrix.txt')
genes = probe_counts['TargetName']
probe_counts = probe_counts.drop(['TargetName', 'ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol',
                                 'Accessions', 'GenomeBuild', 'GenomicPosition', 'GlobalOutlier', 'GlobalOutlierReason',
                                 'OutlierFrequency', 'OutlierAOIs', 'Gene'],1)
probe_counts = probe_counts.rename(index=genes)
probe_counts = probe_counts.astype('int')
negprobe_counts = probe_counts.loc[probe_counts.index == 'NegProbe-WTX', ]
negprobe_counts.index = negprobe_counts.index + '-' + [str(x) for x in range(len(negprobe_counts.index))]
adata = sc.AnnData(X=np.array(counts).T, obs=pd.concat((metadata, properties), axis = 1), var=np.array(counts.index))
adata.var_names = counts.index
adata.obsm['negProbes'] = negprobe_counts.T
adata.obs['GeoLOD2.0_01'] = adata.obs['NegGeoMean_01'] * adata.obs['NegGeoSD_01']**2
adata.obs['DetectedGenes2.0'] = np.sum(np.array([adata.X[i,:] >= adata.obs['GeoLOD2.0_01'].iloc[i] for i in range(len(adata.X[:,1]))]), axis = 1)
adata.var['SYMBOL'] = adata.var[0]
adata.var = adata.var.drop(0, axis = 1)
adata.var.index = [str(x) for x in range(len(adata.var.index))]
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 0] = 5
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 1] = 5
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 2] = 5
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 3] = 5
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 4] = 5

89
['DSP-1001250001588-C-A02', 'DSP-1001250001588-C-A03', 'DSP-1001250001588-C-A04', 'DSP-1001250001588-C-A05', 'DSP-1001250001588-C-A06', 'DSP-1001250001588-C-A07', 'DSP-1001250001588-C-A08', 'DSP-1001250001588-C-A09', 'DSP-1001250001588-C-A10', 'DSP-1001250001588-C-A11', 'DSP-1001250001588-C-A12', 'DSP-1001250001588-C-B01', 'DSP-1001250001588-C-B03', 'DSP-1001250001588-C-B06', 'DSP-1001250001588-C-B09', 'DSP-1001250001588-C-B12', 'DSP-1001250001588-C-C01', 'DSP-1001250001588-C-C03', 'DSP-1001250001588-C-C06', 'DSP-1001250001588-C-C09', 'DSP-1001250001588-C-C12', 'DSP-1001250001588-C-D01', 'DSP-1001250001588-C-D03', 'DSP-1001250001588-C-D06', 'DSP-1001250001588-C-D09', 'DSP-1001250001588-C-D12', 'DSP-1001250001588-C-E01', 'DSP-1001250001588-C-E03', 'DSP-1001250001588-C-E06', 'DSP-1001250001588-C-E09', 'DSP-1001250001588-C-E12', 'DSP-1001250001588-C-F03', 'DSP-1001250001588-C-F06', 'DSP-1001250001588-C-F09', 'DSP-1001250001588-C-F12', 'DSP-1001250001588-C-G03', 'DSP-1001250001588-C-G06

/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [5]:
# Also add previous data:
import pickle
adata_C = pickle.load(open('/nfs/team283/aa16/KR_NAS/data/nanostringWTA_fetailBrain_AnnData.p', 'rb'))
adata_C.obsm['negProbes'] = adata_C.X[:,adata_C.var_names == 'NegProbe-WTX']
adata_C = adata_C[:,adata_C.var_names != 'NegProbe-WTX']
del adata_C.layers['CPM']
del adata_C.layers['X_Corrected_V2']
adata_C.obs['Age'] = adata_C.obs['age']
adata_C.obs['Age'].loc[adata_C.obs['Age'] == '19pcw'] = '19 pcw'
adata_C.obs['Age'].loc[adata_C.obs['Age'] == '14pcw'] = '14 pcw'
adata_C.obs = adata_C.obs.drop('age', axis = 1)
adata_C.obs['slide name'] = adata_C.obs['slide']
adata_C.obs = adata_C.obs.drop('slide', axis = 1)

In [6]:
adata.obs = adata.obs.astype(str)
adataB.obs = adataB.obs.astype(str)
adata_C.obs = adata_C.obs.astype(str)

In [7]:
adata.var_names = adata.var['SYMBOL']
adataB.var_names = adataB.var['SYMBOL']

In [8]:
commonGenes = [g for g in adata_C.var_names if g in np.array(adataB.var_names) ]

In [9]:
adata = adata[:,commonGenes]
adataB = adataB[:,commonGenes]
adata_C = adata_C[:,commonGenes]

In [10]:
adata.obsm['negProbes'] = np.array(adata.obsm['negProbes'])
adataB.obsm['negProbes'] = np.array(adataB.obsm['negProbes'])

In [11]:
adata = anndata.AnnData.concatenate(adata,adataB, adata_C)

In [12]:
adata

AnnData object with n_obs × n_vars = 481 × 17498
    obs: 'Sample_ID', 'Manifest_ID', 'slide name', 'scan name', 'panel', 'roi', 'nuclei', 'segment', 'aoi', 'area', 'Type', 'Species', 'Age', 'Background', 'Tissue', 'Slide code', 'Sample slide ID', 'TR', 'Cy5', 'Cy3', 'Pool', 'Collection plate', 'Well', 'PCR plate', 'PCR well', 'Primer plate', 'Primer well', 'I7_Index_ID', 'index', 'I5_Index_ID', 'index2', 'indexA', 'indexB', 'lane1', 'lane2', 'totalreads', 'Sanger Sample ID', 'Notes', 'expected_neg', 'ns_sample_ID', 'RawReads', 'TrimmedReads', 'StitchedReads', 'AlignedReads', 'DeduplicatedReads', 'SequencingSaturation', 'UID', 'NegGeoMean_01', 'NegGeoSD_01', 'GeoLOQ2.5_01', 'NormFactorNeg_01', 'NormFactorQ3', 'NormFactorHK', 'GeoLOD2.0_01', 'DetectedGenes2.0', 'Manifest ID', 'Tag', 'Reads, count', 'Reads, %', 'ORDER', 'construct', 'instrument_type', 'read_pattern', 'ngs_prep', 'ngs_repnum', 'pcr_primer_plate', 'pcr_primer_well', 'Index 1', 'Barcode 1', 'Index 2', '\tBarcode 2', 'dsp_pl

In [13]:
adata.obs['Donor'] = [str(x).split('-')[0] for x in adata.obs['Sample slide ID']]
(adata.obs['Tissue'].astype(str) + ' ' + adata.obs['Pool'].astype(str)
 + ' ' + adata.obs['Age'].astype(str)).value_counts()

nan nan 19 pcw                                   192
nan nan 14 pcw                                    63
Foetal brain (temporal lobe) CRcells 19 pcw       20
Foetal brain (occipital lobe) CRcells 19 pcw      20
Foetal brain (occipital lobe) Residual 19 pcw     20
Foetal brain (temporal lobe) Residual 19 pcw      19
Foetal brain (frontal lobe) CRcells 19 pcw        18
Foetal brain (frontal lobe) Residual 19 pcw       17
Foetal brain (occipital lobe) Residual 14 pcw     16
Foetal brain (frontal lobe) CRcellsB 14 pcw       16
Foetal brain (occipital lobe) CRcellsB 14 pcw     16
Foetal brain (frontal lobe) Residual 14 pcw       16
Foetal brain (parietal lobe) CRcells 19 pcw       16
Foetal brain (parietal lobe) Residual 19 pcw      14
Foetal brain (temporal lobe) Residual 14 pcw       8
Foetal brain (temporal lobe) CRcellsB 14 pcw       8
nan Residual nan                                   1
Foetal brain (occipital lobe) Rings 19 pcw         1
dtype: int64

In [14]:
adata = adata[adata.obs['slide name'] != 'No Template Control',:]

Save for later use:

In [15]:
adata.obs = adata.obs.astype(str)

In [16]:
adata.write_h5ad('/nfs/team283/aa16/KR_NAS/data/FetalBrain_AllData_AnnData.h5ad')

... storing 'Sample_ID' as categorical
... storing 'Manifest_ID' as categorical
... storing 'slide name' as categorical
... storing 'scan name' as categorical
... storing 'panel' as categorical
... storing 'roi' as categorical
... storing 'nuclei' as categorical
... storing 'segment' as categorical
... storing 'aoi' as categorical
... storing 'area' as categorical
... storing 'Type' as categorical
... storing 'Species' as categorical
... storing 'Age' as categorical
... storing 'Background' as categorical
... storing 'Tissue' as categorical
... storing 'Slide code' as categorical
... storing 'Sample slide ID' as categorical
... storing 'TR' as categorical
... storing 'Cy5' as categorical
... storing 'Cy3' as categorical
... storing 'Pool' as categorical
... storing 'Collection plate' as categorical
... storing 'Well' as categorical
... storing 'PCR plate' as categorical
... storing 'PCR well' as categorical
... storing 'Primer plate' as categorical
... storing 'Primer well' as categori